In [7]:
################################################# Download Senamhi stations for a defined time #########
import os
import pandas as pd

# Tu función process_station
def process_station(file_path, start='1981-01-01', end='2015-12-31'): # time
    date_range = pd.date_range(start=start, end=end, freq='D')
    final_df = pd.DataFrame(index=date_range)
    final_df.index.name = 'date'
    final_df['prec'] = -99.9
    final_df['tmax'] = -99.9
    final_df['tmin'] = -99.9

    with open(file_path, 'r') as f:
        years, months, days = [], [], []
        precs, tmaxs, tmins = [], [], []
        for line in f:
            parts = line.strip().split()
            years.append(int(parts[0]))
            months.append(int(parts[1]))
            days.append(int(parts[2]))
            precs.append(float(parts[3]))
            tmaxs.append(float(parts[4]))
            tmins.append(float(parts[5]))

    data = pd.DataFrame({
        'year': years,
        'month': months,
        'day': days,
        'prec': precs,
        'tmax': tmaxs,
        'tmin': tmins
    })

    data['date'] = pd.to_datetime(data[['year', 'month', 'day']])
    data = data.set_index('date')
    data = data[['prec', 'tmax', 'tmin']]
    final_df.update(data)

    return final_df

# === Files ===
input_root = "D:/S/SENAMHI PERU"
output_root = "D:/S//OUTPUT_CSV_SENAMHI_corrg"  # Puedes cambiar a donde quieras guardar

# === Recorrer todos los departamentos y estaciones ===
for dept in os.listdir(input_root):
    dept_path = os.path.join(input_root, dept)
    if os.path.isdir(dept_path):
        for filename in os.listdir(dept_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(dept_path, filename)
                print(f"Procesando: {file_path}")

                # Procesar estación
                df = process_station(file_path)

                # Crear carpeta de salida si no existe
                output_dir = os.path.join(output_root, dept)
                os.makedirs(output_dir, exist_ok=True)

                # Guardar CSV con mismo nombre de estación
                output_file = os.path.join(output_dir, filename.replace('.txt', '.csv'))
                df.to_csv(output_file)

Procesando: D:/S/SENAMHI PERU\Amazonas\EL PINTOR.txt
Procesando: D:/S/SENAMHI PERU\Amazonas\MAGUNCHAL.txt
Procesando: D:/S/SENAMHI PERU\Ancash\BUENA VISTA.txt
Procesando: D:/S/SENAMHI PERU\Ancash\CHACCHAN.txt
Procesando: D:/S/SENAMHI PERU\Ancash\MAYORARCA.txt
Procesando: D:/S/SENAMHI PERU\Ancash\MILPO.txt
Procesando: D:/S/SENAMHI PERU\Ancash\OCROS.txt
Procesando: D:/S/SENAMHI PERU\Ancash\PIRA.txt
Procesando: D:/S/SENAMHI PERU\Ancash\RECUAY.txt
Procesando: D:/S/SENAMHI PERU\Ancash\SIHUAS.txt
Procesando: D:/S/SENAMHI PERU\Apurimac\CURAHUASI.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\ANDAHUA.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\APLAO.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\AYO.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CARAVELI.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CHACHAS.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CHICHAS.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CHIGUATA.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CHIVAY.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CHOCO.txt

In [9]:
import os
import pandas as pd

# === Directorios ===
input_root = "D:/S/OUTPUT_CSV_SENAMHI_corrg"  # Donde guardaste los CSV de estaciones
output_root = "D:/S/OUTPUT_FACTORES_SENAMHI"  # Nueva carpeta para los factores
os.makedirs(output_root, exist_ok=True)

# Lista para acumular factores
factores = []

# === Función para arreglar nombres mal decodificados (Ã‘ -> Ñ, etc.) ===
def fix_encoding(text):
    try:
        return text.encode("latin1").decode("utf-8")
    except:
        return text

# === Recorrer subcarpetas (departamentos) ===
for dept in os.listdir(input_root):
    dept_path = os.path.join(input_root, dept)
    if os.path.isdir(dept_path):
        for filename in os.listdir(dept_path):
            if filename.endswith(".csv"):
                file_path = os.path.join(dept_path, filename)
                print(f"Procesando: {file_path}")

                # Leer estación con encoding forzado
                df = pd.read_csv(file_path, encoding="latin1")

                # Nombre de estación (desde archivo sin extensión, pero arreglado)
                estacion_name = fix_encoding(filename.replace(".csv", ""))

                # Filtrar datos válidos
                df_prec = df[df['prec'] != -99.9]
                #df_tmax = df[df['tmax'] != -99.9]
                #df_tmin = df[df['tmin'] != -99.9]

                # Calcular promedios (si no hay datos válidos -> None)
                promedio_prec = df_prec['prec'].mean() if not df_prec.empty else None
                #promedio_tmax = df_tmax['tmax'].mean() if not df_tmax.empty else None
                #promedio_tmin = df_tmin['tmin'].mean() if not df_tmin.empty else None

                # Guardar registro
                factores.append({
                    "departamento": dept,
                    "estacion": estacion_name,
                    "promedio_prec": promedio_prec,
                    #"promedio_tmax": promedio_tmax,
                    #"promedio_tmin": promedio_tmin
                })

# === Crear DataFrame con todos los factores ===
factores_df = pd.DataFrame(factores)

# Guardar CSV con los factores
output_file = os.path.join(output_root, "factores_estaciones.csv")
factores_df.to_csv(output_file, index=False, encoding="utf-8-sig")  # UTF-8 con BOM para Excel

print(f"\n✅ Factores (prec, tmax, tmin) guardados en: {output_file} con nombres corregidos")

Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Amazonas\EL PINTOR.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Amazonas\MAGUNCHAL.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Ancash\BUENA VISTA.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Ancash\CHACCHAN.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Ancash\MAYORARCA.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Ancash\MILPO.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Ancash\OCROS.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Ancash\PIRA.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Ancash\RECUAY.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Ancash\SIHUAS.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Apurimac\CURAHUASI.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Arequipa\ANDAHUA.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Arequipa\APLAO.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Arequipa\AYO.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Arequipa\CARAVELI.csv
Procesando: D:/S/OUTPUT_CSV_SENAMHI_corrg\Arequipa\CHACHAS.csv
Procesand

In [16]:
import xarray as xr ########  Download prec, tmax and tmin of grilled data PISCO 
import pandas as pd

# === Rango de coordenadas para todo el Perú ===
longitudes = ['-81.35', '-68.65']  # Oeste a Este
latitudes  = ['-18.35', '-0.05']   # Sur a Norte

# === Función para abrir y procesar un dataset PISCO ===
def load_pisco_dataset(var_url, var_name):
    url = f"""{var_url}\
X/{longitudes[0]}/{longitudes[1]}/RANGEEDGES/\
Y/{latitudes[0]}/{latitudes[1]}/RANGEEDGES/\
dods"""

    ds = xr.open_dataset(url, decode_times=False)
    ds = xr.decode_cf(ds, decode_times=True)
    ds = ds.rename({
        'T': 'time',
        'X': 'lon',
        'Y': 'lat'
    })
    ds['time'] = pd.to_datetime(ds['time'].values.astype(str), yearfirst=True)

    # Cambiar el nombre de la variable interna al que queremos
    var_original = list(ds.data_vars)[0]
    ds = ds.rename({var_original: var_name})

    return ds

# === URLs base para cada variable ===
url_prec_base = "https://iridl.ldeo.columbia.edu/SOURCES/.SENAMHI/.HSR/.PISCO/.Prec/.v2p1/.stable/.daily/.Prec/"
url_tmax_base = "https://iridl.ldeo.columbia.edu/SOURCES/.SENAMHI/.HSR/.PISCO/.Temp/.v1p1/.tmax/.stable/.daily/.tmax/"
url_tmin_base = "https://iridl.ldeo.columbia.edu/SOURCES/.SENAMHI/.HSR/.PISCO/.Temp/.v1p1/.tmin/.stable/.daily/.tmin/"

# === Cargar datasets ===
print("Cargando precipitación...")
ds_prec = load_pisco_dataset(url_prec_base, "prec")

print("Cargando Tmax...")
ds_tmax = load_pisco_dataset(url_tmax_base, "tmax")

print("Cargando Tmin...")
ds_tmin = load_pisco_dataset(url_tmin_base, "tmin")


Cargando precipitación...
Cargando Tmax...
Cargando Tmin...


In [11]:
ds_prec 

<xarray.Dataset>
Dimensions:  (time: 13149, lon: 127, lat: 184)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01T12:00:00 ... 2016-12-31T12:00:00
  * lon      (lon) float32 -81.25 -81.15 -81.05 -80.95 ... -68.85 -68.75 -68.65
  * lat      (lat) float32 -18.35 -18.25 -18.15 -18.05 ... -0.25 -0.15 -0.05
Data variables:
    prec     (time, lat, lon) float32 ...
Attributes:
    Conventions:  IRIDL

In [2]:
import warnings
warnings.filterwarnings("ignore")
import netCDF4
import xarray as xr
import pandas as pd
import glob
import os

In [ ]:
import xarray as xr
import pandas as pd
import os

# Folder containing the era5mmday-YYYY.nc files
input_folder = r"D:/J/era5-land-mmday/Era5land"  ########## OF R

# List all files starting with "era5mmday-" and ending with ".nc"
nc_files = [f for f in os.listdir(input_folder) if f.startswith("era5mmday-") and f.endswith(".nc")]

for nc_file in nc_files:
    # Extract the year from the file name
    year = int(nc_file.split("-")[1].split(".")[0])
    print(f"📂 Processing: {nc_file} (Year {year})")

    # Open the dataset
    ds_path = os.path.join(input_folder, nc_file)
    ds = xr.open_dataset(ds_path)

    # Check if the 'z' dimension exists
    if "z" in ds.dims:
        # Create a real daily time coordinate for this year
        n_days = ds.dims["z"]
        time = pd.date_range(f"{year}-01-01", periods=n_days, freq="D")

        # Rename 'z' to 'time' and assign the new time coordinate
        ds = ds.rename({"z": "time"})
        ds = ds.assign_coords(time=time)

        # Save the corrected dataset with a new name
        output_path = os.path.join("D:/J/era5-land-mmday/Era5land/PERULAND", f"ERA5Land-{year}-mmday.nc")
        ds.to_netcdf(output_path)

        print(f"✅ Saved: {output_path}")
    else:
        print(f"⚠️ Dimension 'z' not found in {nc_file}")

print("🎉 All files have been corrected successfully!")

In [3]:
folder_path= "D:/J/era5-land-mmday/Era5land/PERULAND/"

In [4]:
# Find all netcdf4 files in the folder
file_list = glob.glob(folder_path + "*.nc")
file_list

['D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1981-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1982-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1983-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1984-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1985-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1986-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1987-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1988-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1989-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1990-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1991-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1992-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1993-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1994-mmday.nc',
 'D:/J/era5-land-mmday/Era5land/PERULAND\\ERA5Land-1995-mmday.

In [5]:
#load all netcdf files in the folder and combine them by time
ds = xr.open_mfdataset(file_list, combine = "nested", concat_dim = "time")

In [6]:
ds

<xarray.Dataset>
Dimensions:    (time: 16071, longitude: 131, latitude: 190)
Coordinates:
  * longitude  (longitude) float64 -81.5 -81.4 -81.3 -81.2 ... -68.7 -68.6 -68.5
  * latitude   (latitude) float64 -0.1 -0.2 -0.3 -0.4 ... -18.8 -18.9 -19.0
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2024-12-31
Data variables:
    crs        (time) int32 -2147483647 -2147483647 ... -2147483647 -2147483647
    ppt        (time, latitude, longitude) float32 dask.array<chunksize=(365, 190, 131), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-30)
    date:         2025-07-23 23:05:08

In [7]:
ds.to_netcdf("D:/X/LAND-PERU/LANDPERU7.nc")

In [8]:
import xarray as xr
ds = xr.open_dataset("D:/X/LAND-PERU/LANDPERU7.nc")
print(ds)

<xarray.Dataset>
Dimensions:    (time: 16071, longitude: 131, latitude: 190)
Coordinates:
  * longitude  (longitude) float64 -81.5 -81.4 -81.3 -81.2 ... -68.7 -68.6 -68.5
  * latitude   (latitude) float64 -0.1 -0.2 -0.3 -0.4 ... -18.8 -18.9 -19.0
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2024-12-31
Data variables:
    crs        (time) int32 ...
    ppt        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-30)
    date:         2025-07-23 23:05:08


In [ ]:
import xarray as xr    ##  okkkk
import numpy as np

input_path = "D:/X/LAND-PERU/LANDPERU7.nc"
output_path = "D:/R//LANDPERU7km.nc"

# Abrir dataset
ds = xr.open_dataset(input_path)

# Asegurar nombres de coordenadas
ds = ds.rename({
    'latitude': 'lat' if 'latitude' in ds.dims else 'lat',
    'longitude': 'lon' if 'longitude' in ds.dims else 'lon'
})

# Asegurar orden de lat (de norte a sur)
if ds['lat'][0] < ds['lat'][-1]:
    ds = ds.reindex(lat=ds.lat[::-1])

# Transponer la variable principal
if 'ppt' in ds:
    ds['ppt'] = ds['ppt'].transpose('time', 'lat', 'lon')

# Asegurar atributos CF estándar
ds['lat'].attrs = {
    'units': 'degrees_north',
    'long_name': 'latitude',
    'standard_name': 'latitude'
}
ds['lon'].attrs = {
    'units': 'degrees_east',
    'long_name': 'longitude',
    'standard_name': 'longitude'
}
ds['time'].attrs = {
    'long_name': 'time',
    'standard_name': 'time'
}
ds['ppt'].attrs['units'] = 'mm/day'

# Guardar NetCDF compatible
ds.to_netcdf(output_path)
print(f"✅ Archivo guardado en formato ArcGIS-friendly: {output_path}")

In [28]:
ds_precland = xr.open_dataset("D:/R//LANDPERU7km.nc")
print(ds_precland)

<xarray.Dataset>
Dimensions:  (time: 16071, lon: 131, lat: 190)
Coordinates:
  * lon      (lon) float64 -81.5 -81.4 -81.3 -81.2 ... -68.8 -68.7 -68.6 -68.5
  * lat      (lat) float64 -0.1 -0.2 -0.3 -0.4 -0.5 ... -18.7 -18.8 -18.9 -19.0
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2024-12-31
Data variables:
    crs      (time) int32 ...
    ppt      (time, lat, lon) float32 ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-30)
    date:         2025-07-23 23:05:08


In [29]:
import polars as pl  #!pip install polars

# Try reading with Latin-1 encoding (Windows default)
ubicaciones_df = pl.read_csv(
    "D:/S/SENAMHI PERU/Stations Senamhi.csv", 
    encoding="latin1"
)[["Stations", "Lat", "Lon"]]

# Convert to list of dictionaries
locs = ubicaciones_df.to_dicts()

# Preview the first few locations
print(locs[:5])

[{'Stations': 'CABANILLAS', 'Lat': -15.16, 'Lon': -70.0}, {'Stations': 'CAÑETE', 'Lat': -13.07, 'Lon': -76.3}, {'Stations': 'CHOTANO LAJAS', 'Lat': -6.56, 'Lon': -78.7}, {'Stations': 'LA CASCARILLA', 'Lat': -5.66, 'Lon': -78.9}, {'Stations': 'ÑAÑA', 'Lat': -11.99, 'Lon': -76.9}]


In [30]:
# === Supongamos que ya tienes cargados ds_prec, ds_tmax, ds_tmin ===
prec = xr.Dataset()
tmax = xr.Dataset()
tmin = xr.Dataset()
precland = xr.Dataset() ######## predictor

for l in locs:
    nombre = l['Stations']
    lon = l['Lon']
    lat = l['Lat']

    # --- Precipitación ---
    ds_prec_sel = ds_prec.sel(lon=lon, lat=lat, method='nearest')
    ds_prec_sel.attrs[f"{nombre}_lon"] = float(ds_prec_sel.lon.values)
    ds_prec_sel.attrs[f"{nombre}_lat"] = float(ds_prec_sel.lat.values)
    ds_prec_sel = ds_prec_sel.rename({"prec": nombre}).drop_vars(["lat", "lon"])
    prec = xr.merge([prec, ds_prec_sel], compat="override")

    # --- Tmax ---
    ds_tmax_sel = ds_tmax.sel(lon=lon, lat=lat, method='nearest')
    ds_tmax_sel.attrs[f"{nombre}_lon"] = float(ds_tmax_sel.lon.values)
    ds_tmax_sel.attrs[f"{nombre}_lat"] = float(ds_tmax_sel.lat.values)
    ds_tmax_sel = ds_tmax_sel.rename({"tmax": nombre}).drop_vars(["lat", "lon"])
    tmax = xr.merge([tmax, ds_tmax_sel], compat="override")

    # --- Tmin ---
    ds_tmin_sel = ds_tmin.sel(lon=lon, lat=lat, method='nearest')
    ds_tmin_sel.attrs[f"{nombre}_lon"] = float(ds_tmin_sel.lon.values)
    ds_tmin_sel.attrs[f"{nombre}_lat"] = float(ds_tmin_sel.lat.values)
    ds_tmin_sel = ds_tmin_sel.rename({"tmin": nombre}).drop_vars(["lat", "lon"])
    tmin = xr.merge([tmin, ds_tmin_sel], compat="override")

    # --- Era5land ---
    ds_precland_sel = ds_precland.sel(lon=lon, lat=lat, method='nearest')
    ds_precland_sel.attrs[f"{nombre}_lon"] = float(ds_precland_sel.lon.values)
    ds_precland_sel.attrs[f"{nombre}_lat"] = float(ds_precland_sel.lat.values)
    ds_precland_sel = ds_precland_sel.rename({"ppt": nombre}).drop_vars(["lat", "lon"])
    precland = xr.merge([precland, ds_precland_sel], compat="override")

# Si luego quieres guardar comprimido:
# prec.to_netcdf("prec_estaciones.nc", encoding={v: {"zlib": True, "complevel": 4} for v in prec.data_vars})
prec.data_vars

Data variables:
    CABANILLAS                  (time) float32 ...
    CAÑETE                      (time) float32 ...
    CHOTANO LAJAS               (time) float32 ...
    LA CASCARILLA               (time) float32 ...
    ÑAÑA                        (time) float32 ...
    HUANGACOCHA                 (time) float32 ...
    AYAVIRI-PUN                 (time) float32 ...
    MUÑANI                      (time) float32 ...
    FRANCISCO DE ORELLANA       (time) float32 ...
    MANITI                      (time) float32 ...
    BELLAVISTA                  (time) float32 ...
    CHILCAYOC                   (time) float32 ...
    PAUZA                       (time) float32 ...
    PAMPA BLANCA-ARE            (time) float32 ...
    CAPACHICA                   (time) float32 ...
    PAMPAHUTA                   (time) float32 ...
    LAMPA                       (time) float32 ...
    CARAVELI                    (time) float32 ...
    YANAQUIHUA                  (time) float32 ...
    LAMPA-AYA  

In [45]:
import pandas as pd     #### it may take a few minutes

# === Diccionario final ===
#clima_df_dict = {}

# Crear diccionarios base
prec_df_dict = {}
tmax_df_dict = {}
tmin_df_dict = {}
precland_df_dict = {}###########  predictor

# --- Precipitación ---
for var_name in prec.data_vars:
    serie = prec[var_name].to_series()
    df = serie.to_frame(name='prec')
    prec_df_dict[var_name.strip()] = df

# --- Tmax ---
for var_name in tmax.data_vars:
    serie = tmax[var_name].to_series()
    df = serie.to_frame(name='tmax')
    tmax_df_dict[var_name.strip()] = df

# --- Tmin ---
for var_name in tmin.data_vars:
    serie = tmin[var_name].to_series()
    df = serie.to_frame(name='tmin')
    tmin_df_dict[var_name.strip()] = df

# --- Preland ---
for var_name in precland.data_vars:
    serie = precland[var_name].to_series()
    df = serie.to_frame(name='ppt')
    precland_df_dict[var_name.strip()] = df

# --- Unir los tres en un solo diccionario ---
#for estacion in prec_df_dict.keys():
    #df_prec = prec_df_dict[estacion]
    #df_tmax = tmax_df_dict.get(estacion, pd.DataFrame())
    #df_tmin = tmin_df_dict.get(estacion, pd.DataFrame())
    #df_precland = precland_df_dict.get(estacion, pd.DataFrame())

    # Unir por índice (tiempo)
    #df_merged = df_prec.join(df_tmax, how="outer").join(df_tmin, how="outer").join(df_precland, how="outer")

    #clima_df_dict[estacion] = df_merged

# clima_df_dict ahora tiene un DataFrame por estación con columnas: prec, tmax, tmin
#clima_df_dict['NOMBRE_ESTACION']

In [44]:
precland_df_dict ['EL LIMON']

KeyError: 'EL LIMON'

In [15]:
clima_df_dict['EL PINTOR']

,prec,tmax,tmin
time,,,
1981-01-01 12:00:00,0.023738,32.935612,18.684752
1981-01-02 12:00:00,0.906146,32.917114,19.544960
1981-01-03 12:00:00,1.185733,32.146633,20.293718
1981-01-04 12:00:00,0.059542,33.327045,19.230915
1981-01-05 12:00:00,0.000000,33.325817,19.627443
...,...,...,...
2016-12-27 12:00:00,1.372402,34.543102,21.843784
2016-12-28 12:00:00,2.860460,31.457262,22.051888
2016-12-29 12:00:00,1.118405,31.511045,21.946974


In [64]:
import os
import pandas as pd

# === Rutas ===
factores_obs_file = "D:/S/OUTPUT_FACTORES_SENAMHI/factores_estaciones.csv"
output_file = "D:/S/OUTPUT_FACTORES_prec_SENAMHI/factores_vs_pisco.csv"

# === Función para arreglar nombres mal decodificados (Ã‘ -> Ñ, etc.) ===
def fix_encoding(text):
    try:
        return text.encode("latin1").decode("utf-8")
    except:
        return text

# === Cargar promedios observados ===
obs_df = pd.read_csv(factores_obs_file, encoding="utf-8-sig")
# Corregir nombres de estaciones
obs_df["estacion_fixed"] = obs_df["estacion"].apply(lambda x: fix_encoding(str(x)).strip().upper())

# Definir rango válido (igual al de observaciones)
start_date = "1981-01-01"
end_date   = "2015-12-31"

# Lista para guardar resultados
resultados = []

# === Recorrer estaciones en PISCO (solo prec) ===
for estacion, df in prec_df_dict.items():
    # Arreglar nombre de estación desde PISCO
    estacion_fixed = fix_encoding(str(estacion)).strip().upper()

    # Asegurar que el índice sea datetime
    df = df.copy()
    df.index = pd.to_datetime(df.index)

    # Filtrar solo 1981–2015
    df_periodo = df.loc[start_date:end_date]

    # Calcular promedio PISCO solo en ese rango
    promedio_pisco_prec = df_periodo['prec'].mean(skipna=True)

    # Buscar observados correspondientes usando estacion_fixed
    obs_row = obs_df[obs_df['estacion_fixed'] == estacion_fixed]

    if not obs_row.empty:
        promedio_obs_prec = obs_row['promedio_prec'].values[0]

        # Calcular factor (evitar división por cero)
        factor_prec = promedio_obs_prec / promedio_pisco_prec if promedio_pisco_prec and promedio_pisco_prec != 0 else None

        resultados.append({
            "estacion": fix_encoding(estacion),  # nombre original corregido
            "promedio_obs_prec": promedio_obs_prec,
            "promedio_pisco_prec": promedio_pisco_prec,
            "factor_prec": factor_prec
        })

# === Convertir a DataFrame y redondear a 2 decimales ===
factores_final_df = pd.DataFrame(resultados).round(2)

# Guardar CSV con 2 decimales y UTF-8 con BOM para que Excel muestre Ñ correctamente
factores_final_df.to_csv(output_file, index=False, float_format="%.2f", encoding="utf-8-sig")

print(f"\n✅ Factores PISCO vs Observado guardados en: {output_file} (1981–2015, con 2 decimales y nombres corregidos)")



✅ Factores PISCO vs Observado guardados en: D:/S/OUTPUT_FACTORES_prec_SENAMHI/factores_vs_pisco.csv (1981–2015, con 2 decimales y nombres corregidos)


In [66]:
import os ###################### coregir para prec
import pandas as pd

# === Rutas ===
factores_file = "D:/S/OUTPUT_FACTORES_SENAMHI/factores_vs_pisco.csv"

# === Cargar factores ===
factores_df = pd.read_csv(factores_file, encoding="utf-8-sig")

# Diccionario para guardar datos corregidos
prec_df_corr = {}  

# === Recorrer estaciones de prec_df_dict ===
for estacion, df in prec_df_dict.items():  
    df_corr = df.copy()

    # Buscar factores de esa estación (coincidencia insensible a mayúsculas)
    row = factores_df[factores_df["estacion"].str.upper() == estacion.upper()]

    if not row.empty:
        factor_prec = row["factor_prec"].values[0]

        # ✅ Aplicar corrección SOLO a prec
        if pd.notna(factor_prec):
            df_corr["prec"] = df_corr["prec"] * factor_prec
            print(f"✅ Estación {estacion}: factor_prec aplicado ({factor_prec})")
    else:
        print(f"⚠️ Estación {estacion} no encontrada en factores, se deja sin corrección.")

    # Guardar en el diccionario corregido
    prec_df_corr[estacion] = df_corr


✅ Estación CABANILLAS: factor_prec aplicado (1.07)
✅ Estación CAÑETE: factor_prec aplicado (2.86)
✅ Estación CHOTANO LAJAS: factor_prec aplicado (1.1)
✅ Estación LA CASCARILLA: factor_prec aplicado (2.46)
✅ Estación ÑAÑA: factor_prec aplicado (0.47)
✅ Estación HUANGACOCHA: factor_prec aplicado (4.25)
✅ Estación AYAVIRI-PUN: factor_prec aplicado (1.01)
✅ Estación MUÑANI: factor_prec aplicado (1.01)
✅ Estación FRANCISCO DE ORELLANA: factor_prec aplicado (1.55)
✅ Estación MANITI: factor_prec aplicado (1.36)
✅ Estación BELLAVISTA: factor_prec aplicado (1.14)
✅ Estación CHILCAYOC: factor_prec aplicado (1.68)
✅ Estación PAUZA: factor_prec aplicado (1.17)
✅ Estación PAMPA BLANCA-ARE: factor_prec aplicado (3.4)
✅ Estación CAPACHICA: factor_prec aplicado (0.87)
✅ Estación PAMPAHUTA: factor_prec aplicado (1.03)
✅ Estación LAMPA: factor_prec aplicado (1.2)
✅ Estación CARAVELI: factor_prec aplicado (1.2)
✅ Estación YANAQUIHUA: factor_prec aplicado (1.65)
✅ Estación LAMPA-AYA: factor_prec aplicado 

In [12]:
prec_df_corr['EL PINTOR']
# Seleccionar la estación
df_el_pintor = prec_df_corr["EL PINTOR"]

# Exportar a CSV
df_el_pintor.to_csv("D:/S/EL_PINTOR_prec.csv", float_format="%.2f")

print("✅ Archivo exportado: D:/S/EL_PINTOR_prec.csv")


✅ Archivo exportado: D:/S/EL_PINTOR_prec.csv


In [67]:
################################# Complete missing data Senamhi with PISCO + add Precland ##############################
import os
import pandas as pd
from pathlib import Path

# Rutas base
input_base = Path("D:/S/OUTPUT_CSV_SENAMHI_corrg")
output_base = Path("D:/S/OUTPUT_ERA5LAND_SENAMHI_NC_corr")

# Normalizar fechas en diccionarios
for dic in (prec_df_corr, tmax_df_dict, tmin_df_dict, precland_df_dict):
    for key in dic:
        df = dic[key]
        df.index = df.index.normalize()
        dic[key] = df

# Procesar cada archivo CSV
for dept_path in input_base.iterdir():
    if dept_path.is_dir():
        dept_name = dept_path.name
        for station_file in dept_path.glob("*.csv"):
            station_name = station_file.stem
            print(f"Procesando: {dept_name}/{station_name}")

            # Leer CSV original
            df_csv = pd.read_csv(station_file, index_col=0, parse_dates=True)

            # Verificar si la estación existe en los datos de NetCDF
            if (
                station_name in prec_df_corr
                and station_name in tmax_df_dict
                and station_name in tmin_df_dict
                and station_name in precland_df_dict
            ):
                df_prec_nc = prec_df_corr[station_name]
                df_tmax_nc = tmax_df_dict[station_name]
                df_tmin_nc = tmin_df_dict[station_name]
                df_precland_nc = precland_df_dict[station_name]

                # Asegurarse de que las fechas coincidan
                common_dates = df_csv.index.intersection(df_prec_nc.index)

                # --- Reemplazar PREC ---
                df_csv.loc[common_dates, "prec"] = df_csv.loc[common_dates, "prec"].mask(
                    df_csv.loc[common_dates, "prec"] == -99.9,
                    df_prec_nc.loc[common_dates, "prec"].round(1)
                )

                # --- Reemplazar TMAX ---
                if "tmax" in df_csv.columns:
                    df_csv.loc[common_dates, "tmax"] = df_csv.loc[common_dates, "tmax"].mask(
                        df_csv.loc[common_dates, "tmax"] == -99.9,
                        df_tmax_nc.loc[common_dates, "tmax"].round(1)
                    )

                # --- Reemplazar TMIN ---
                if "tmin" in df_csv.columns:
                    df_csv.loc[common_dates, "tmin"] = df_csv.loc[common_dates, "tmin"].mask(
                        df_csv.loc[common_dates, "tmin"] == -99.9,
                        df_tmin_nc.loc[common_dates, "tmin"].round(1)
                    )

                # --- Agregar columna PRECLAND ---
                df_csv["precland"] = df_precland_nc.reindex(df_csv.index)["ppt"].round(1)

                # Guardar en carpeta de salida
                output_dept_dir = output_base / dept_name
                output_dept_dir.mkdir(parents=True, exist_ok=True)
                output_path = output_dept_dir / f"{station_name}.csv"
                df_csv.to_csv(output_path)

            else:
                print(f"⚠️ Estación no encontrada en NetCDF o precland: {station_name}")


Procesando: Amazonas/EL PINTOR
Procesando: Amazonas/MAGUNCHAL
Procesando: Ancash/BUENA VISTA
Procesando: Ancash/CHACCHAN
Procesando: Ancash/MAYORARCA
Procesando: Ancash/MILPO
Procesando: Ancash/OCROS
Procesando: Ancash/PIRA
Procesando: Ancash/RECUAY
Procesando: Ancash/SIHUAS
Procesando: Apurimac/CURAHUASI
Procesando: Arequipa/ANDAHUA
Procesando: Arequipa/APLAO
Procesando: Arequipa/AYO
Procesando: Arequipa/CARAVELI
Procesando: Arequipa/CHACHAS
Procesando: Arequipa/CHICHAS
Procesando: Arequipa/CHIGUATA
Procesando: Arequipa/CHIVAY
Procesando: Arequipa/CHOCO
Procesando: Arequipa/COTAHUASI
Procesando: Arequipa/EL FRAYLE
Procesando: Arequipa/HUAMBO
Procesando: Arequipa/IMATA
Procesando: Arequipa/LA ANGOSTURA
Procesando: Arequipa/LA JOYA
Procesando: Arequipa/LA PAMPILLA
Procesando: Arequipa/LAS SALINAS
Procesando: Arequipa/MACHAGUAY
Procesando: Arequipa/MADRIGAL
Procesando: Arequipa/ORCOPAMPA
Procesando: Arequipa/PAMPA BLANCA-ARE
Procesando: Arequipa/PAMPA DE MAJES
Procesando: Arequipa/PILLON

In [1]:
from pathlib import Path ############# of all files except some subfolders
import shutil

# Carpeta base con subcarpetas de departamentos
output_base = Path("D:/S/OUTPUT_ERA5LAND_SENAMHI_NC_corr")

# Carpeta donde juntarás todos los CSV excepto los de una carpeta
consolidated_dir = Path("D:/S/OUTPUT_CSV_SENAMHI_NC_ALL") ############ do it whit this.
consolidated_dir.mkdir(exist_ok=True)

# Nombre de la carpeta que quieres excluir
carpetas_excluir = {"Loreto", "San Martin"} # aut of study area

# Recorrer todas las subcarpetas y copiar archivos que NO sean de la carpeta excluida
for csv_file in output_base.rglob("*.csv"):
    # Obtener nombre de la subcarpeta (departamento)
    relative_parts = csv_file.relative_to(output_base).parts
    if relative_parts[0] in carpetas_excluir:
        continue # saltar esta carpeta

    # Copiar archivo
    destination = consolidated_dir / csv_file.name
    shutil.copy(csv_file, destination)

print(f"✅ Todos los CSV fueron consolidados en la carpeta OUTPUT_CSV_SENAMHI_NC_ALL (excepto '{carpetas_excluir}')")


✅ Todos los CSV fueron consolidados en la carpeta OUTPUT_CSV_SENAMHI_NC_ALL (excepto '{'Loreto', 'San Martin'}')


In [68]:
from pathlib import Path  ###############  for some subfolders except the others
import shutil

# Carpeta base con subcarpetas por departamento
output_base = Path("D:/S/OUTPUT_ERA5LAND_SENAMHI_NC_corr")

# Carpeta destino donde guardarás los CSV seleccionados
consolidated_dir = Path("D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_Lima_corrg")
consolidated_dir.mkdir(parents=True, exist_ok=True)

# Subcarpetas que SÍ quieres incluir
carpetas_incluir = {"Lima"}  # usa el nombre exacto de las carpetas {"Cusco", "Puno"}  

# Copiar archivos de las carpetas seleccionadas, conservando el nombre original
for carpeta in carpetas_incluir:
    carpeta_path = output_base / carpeta
    if not carpeta_path.exists():
        print(f"⚠️ Carpeta no encontrada: {carpeta_path}")
        continue

    for csv_file in carpeta_path.glob("*.csv"):
        destino = consolidated_dir / csv_file.name  # usa el nombre original
        shutil.copy(csv_file, destino)
        #print(f"Copiado: {csv_file} → {destino}")

print("✅ Todos los CSV fueron copiados con su nombre original desde:", ", ".join(carpetas_incluir))

✅ Todos los CSV fueron copiados con su nombre original desde: Lima


In [69]:
import pandas as pd ################################## ramdom forest ##################################################
import os
from glob import glob
import unidecode
from datetime import timedelta

# === 1. Cargar coordenadas y datos estáticos ===
coord_df = pd.read_csv("D:/S/SENAMHI PERU/Stations Senamhi.csv", encoding="latin1")
coord_df["station_clean"] = coord_df["Stations"].str.strip().str.upper().apply(unidecode.unidecode)
coord_df = coord_df.sort_values("station_clean").reset_index(drop=True)

# Añadir staid (ID por estación)
#coord_df["staid"] = range(1, len(coord_df) + 1)

# Diccionario con todas las variables
coord_dict = coord_df.set_index("station_clean").to_dict(orient="index")

# === 2. Leer archivos de estaciones ===
folder = "D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_Lima_corrg"
files = glob(os.path.join(folder, "*.csv"))

# Fechas deseadas
start_date = pd.to_datetime("1981-01-01")
end_date = pd.to_datetime("1983-12-31")

# === 3. Procesar y agrupar por fecha ===
station_data = []

for file in files:
    station = os.path.basename(file).replace(".csv", "")
    station_clean = unidecode.unidecode(station.strip().upper())

    if station_clean not in coord_dict:
        print(f"⚠️ Estación {station} no está en coordenadas.")
        continue

    try:
        df = pd.read_csv(file)

        if "date" not in df.columns or "prec" not in df.columns:
            print(f"⚠️ {station}: columnas 'date' o 'prec' no están presentes.")
            continue

        df["date"] = pd.to_datetime(df["date"], errors="coerce")
        df = df.dropna(subset=["date"])
        df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]

        # Datos estáticos de la estación
        info = coord_dict[station_clean]
        lon, lat = info["Lon"], info["Lat"]

        for _, row in df.iterrows():
            current_date = row["date"]
            next_date = current_date + timedelta(days=1)

            station_data.append({
                "lon": lon,
                "lat": lat,
                "time": current_date,
                "endTime": next_date,
                "prcp": row["prec"],
                "tmax": row.get("tmax", None),
                "tmin": row.get("tmin", None),
                "precland": row.get("precland", None),   # 👈✅ NUEVA COLUMNA
                #"staid": info["staid"],
                #"h": info["sin"],        # si 'h' es sin, ajustar si no
                "dem": info["dem"],
                "twi": info["twi"],
                "sin": info["sin"],
                "cos": info["cos"]
            })

    except Exception as e:
        print(f"⚠️ Error procesando {station}: {e}")

# === 4. Crear DataFrame final ===
df_all = pd.DataFrame(station_data)
df_all = df_all.sort_values(by=["time", "lat", "lon"]).reset_index(drop=True)

# sp.ID reinicia cada fecha
df_all["sp.ID"] = df_all.groupby("time").cumcount() + 1

# timeIndex (contador de días desde start_date)
df_all["timeIndex"] = (df_all["time"] - start_date).dt.days + 1

# Formato de fechas
df_all["time"] = df_all["time"].dt.strftime("%m/%d/%Y")
df_all["endTime"] = df_all["endTime"].dt.strftime("%m/%d/%Y")

# === 5. Ordenar columnas ===
column_order = [
    "lon", "lat", "sp.ID", "time", "endTime", "timeIndex",
    "dem", "twi", "prcp", "tmax", "tmin", "precland", "sin", "cos"
]
df_all = df_all[column_order]
# === 5. Guardar ===
output_path = "D:/S/Serbia1km/Interpolation/stfdf_pt_lima.csv"
df_all.to_csv(output_path, index=False, encoding="utf-8")
print("✅ Archivo generado:", output_path)

✅ Archivo generado: D:/S/Serbia1km/Interpolation/stfdf_pt_lima.csv


In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

# Activar la conversión automática entre pandas y R
pandas2ri.activate()

# Pasar df_all a R
ro.globalenv["df_all_r"] = pandas2ri.py2rpy(df_all)

# Guardar en formato .rda
rda_path = "D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNO/stfdf_pt_puno.rda"
ro.r(f'save(df_all_r, file="{rda_path}")')

print("✅ Archivo guardado en formato .rda:", rda_path)


In [ ]:
import polars as pl

# Convertir el DataFrame final de pandas a polars
df_all_pl = pl.from_pandas(df_all)

# Guardar en formato Parquet
output_path_parquet = "D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNO/stfdf_pt_puno.parquet"
df_all_pl.write_parquet(output_path_parquet)

print("✅ Archivo Parquet generado:", output_path_parquet)

In [ ]:
# 1. Cargar librerías
import xarray as xr
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import shape
import regionmask
# 2. Cargar los archivos netCDF
ds1 = xr.open_dataset('F:/Proyecto_002/data/pisco/vf/obs_pisco_pp.nc')
ds2 = xr.open_dataset('F:/Proyecto_002/data/pr/vf/pr_ACCESSCM2_1981_2014.nc')
# 3. Calcular los índices estadísticos
correlation = xr.corr(ds1['pr'], ds2['pr'], dim='time')
RMSE = xs.rmse(ds1['pr'], ds2['pr'], dim='time')
pBIAS = calculate_pbias(ds1['pr'], ds2['pr'])
# 4. Crear los subplots en una matriz 5 x 4 y configurar el estilo
plt.rcParams['font.family'] = 'Times New Roman'
fig, axs = plt.subplots(4, 6, figsize=(24, 15))
w=1
# 5. Plotear cada gráfico en la matriz de ejes
for i in range(4):
 for j in range(6):
 ax = axs[i, j]
 print(w, model_lista[w-1], lista_modelos[w-1])
 ds2 = xr.open_dataset(model_lista[w-1])
 indice_r2 = xr.corr(ds1['pr'], ds2['pr'], dim='time')
 dataplot = indice_r2.where(mask_f)
 cmap_reversed = plt.cm.get_cmap('turbo').reversed(
 img = dataplot.plot(ax=ax, cmap=cmap_reversed,
add_colorbar=False)
 gdf.plot(ax=ax, facecolor='none', edgecolor='red')
 ax.axis('off')
 model_name = lista_modelos[w-1] # Obtener el nombre del modelo
 ax.text(0.5, -0.15, model_name, transform=ax.transAxes,
fontsize=25, ha='center') # Nombre del modelo abajo
 ds2.close()
 w=w+1

In [21]:
import os######################### of QMAP
import pandas as pd
import glob

# --- Rutas ---
base_dir = r"D:/S/OUTPUT_ERA5LAND_SENAMHI_NC_corr"
stations_file = r"D:/S/SENAMHI PERU/Stations Senamhi.csv"
output_dir = r"D:/S/CDBC"

# Crear carpeta Obs si no existe
os.makedirs(output_dir, exist_ok=True)

# --- Leer info de estaciones ---
ubicaciones_df = pd.read_csv(stations_file, encoding="latin1")
ubicaciones_df = ubicaciones_df.rename(columns={"Stations":"NAME","Lat":"LAT","Lon":"LONG","dem":"ELEVATION"})

# --- Diccionario de series ---
series_dict = {}
latitudes, longitudes = [], []

# --- Buscar CSV de estaciones ---
for _, row in ubicaciones_df.iterrows():
    est = row["NAME"]
    lat, lon = row["LAT"], row["LONG"]

    # Buscar archivo CSV de la estación en todas las subcarpetas
    pattern = os.path.join(base_dir, "**", f"{est}.csv")
    matches = glob.glob(pattern, recursive=True)
    if not matches:
        print(f"⚠️ No se encontró archivo para {est}")
        continue

    file_path = matches[0]
    df = pd.read_csv(file_path)

    if "date" not in df.columns or "precland" not in df.columns: ######### Download Obs (prec) or hist( precland crudo)
        print(f"⚠️ {est}: CSV sin columnas esperadas")
        continue

    # Serie de precipitaciones
    df["date"] = pd.to_datetime(df["date"])
    serie = df.set_index("date")["precland"]

    # Clave: nombre de estación (para mantener orden)
    series_dict[est] = serie.round(2)
    latitudes.append(lat)
    longitudes.append(lon)

# --- Unir todas las series ---
if series_dict:
    final_df = pd.concat(series_dict, axis=1).sort_index()
    final_df = final_df.reset_index()
    final_df["date"] = final_df["date"].dt.strftime("%#d/%#m/%Y")

    # Reemplazar nombres de estaciones por nada (solo lat/lon arriba)
    final_df.columns = ["Fecha"] + ["" for _ in range(len(final_df.columns) - 1)]

    # Preparar encabezados de lat y lon
    header_lat = pd.DataFrame([["N"] + latitudes], columns=final_df.columns)
    header_lon = pd.DataFrame([["E"] + longitudes], columns=final_df.columns)

    # Concatenar encabezados + datos
    df_out = pd.concat([header_lat, header_lon, final_df], ignore_index=True)

    # Guardar CSV final
    # Guardar CSV en la carpeta Obs
    salida = os.path.join(output_dir, "ModH.csv")
    df_out.to_csv(salida, index=False, header=False)
    print(f"✅ Guardado en {salida}")
else:
    print("❌ No se encontraron datos")



✅ Guardado en D:/S/CDBC\ModH.csv


In [51]:
import os ####################### just of Lima
import pandas as pd
import glob

# --- Rutas ---
base_dir = r"D:/S/OUTPUT_CSV_SENAMHI_NC"## para pisco completado
stations_file = r"D:/S/SENAMHI PERU/Stations Senamhi.csv"
output_dir = r"D:/S/CDBC"
departamento_filtro = "Lima"

os.makedirs(output_dir, exist_ok=True)

# --- Leer info de estaciones ---
ubicaciones_df = pd.read_csv(stations_file, encoding="latin1")
ubicaciones_df = ubicaciones_df.rename(columns={"Stations":"NAME","Lat":"LAT","Lon":"LONG","dem":"ELEVATION"})

# --- Filtrar estaciones solo del departamento deseado ---
ubicaciones_df = ubicaciones_df[ubicaciones_df["NAME"].notna()]  # asegurar que NAME no sea NA

# --- Diccionario de series ---
series_dict = {}
latitudes, longitudes = [], []

# --- Recorrer todos los CSV dentro de la subcarpeta del departamento ---
dept_dir = os.path.join(base_dir, departamento_filtro)
csv_files = glob.glob(os.path.join(dept_dir, "*.csv"))

if not csv_files:
    print(f"❌ No se encontraron archivos CSV en {dept_dir}")
else:
    for file_path in csv_files:
        est = os.path.splitext(os.path.basename(file_path))[0]  # nombre de la estación según el archivo
        # Verificar si la estación está en tu tabla de ubicaciones
        if est not in ubicaciones_df["NAME"].values:
            print(f"⚠️ {est} no está en stations_file, se salta")
            continue

        row = ubicaciones_df[ubicaciones_df["NAME"] == est].iloc[0]
        lat, lon = row["LAT"], row["LONG"]

        df = pd.read_csv(file_path)
        if "date" not in df.columns or "prec" not in df.columns:
            print(f"⚠️ {est}: CSV sin columnas esperadas")
            continue

        df["date"] = pd.to_datetime(df["date"])
        serie = df.set_index("date")["prec"]

        series_dict[est] = serie.round(2)
        latitudes.append(lat)
        longitudes.append(lon)

    # --- Unir todas las series ---
    if series_dict:
        final_df = pd.concat(series_dict, axis=1).sort_index()
        final_df = final_df.reset_index()
        final_df["date"] = final_df["date"].dt.strftime("%#d/%#m/%Y")

        final_df.columns = ["Fecha"] + ["" for _ in range(len(final_df.columns) - 1)]

        header_lat = pd.DataFrame([["N"] + latitudes], columns=final_df.columns)
        header_lon = pd.DataFrame([["E"] + longitudes], columns=final_df.columns)

        df_out = pd.concat([header_lat, header_lon, final_df], ignore_index=True)

        salida = os.path.join(output_dir, f"ObsHpisc_{departamento_filtro}.csv")
        df_out.to_csv(salida, index=False, header=False)
        print(f"✅ Guardado en {salida}")
    else:
        print("❌ No se encontraron datos válidos")


⚠️ MATUCANA no está en stations_file, se salta
⚠️ SANTA CRUZ no está en stations_file, se salta
✅ Guardado en D:/S/CDBC\ObsHpisc_Lima.csv


In [49]:
import os
import pandas as pd

# --- Rutas ---
stations_file = r"D:/S/SENAMHI PERU/Stations Senamhi.csv"
output_dir = r"D:/S/CDBC"
os.makedirs(output_dir, exist_ok=True)

# --- Leer info de estaciones ---
ubicaciones_df = pd.read_csv(stations_file, encoding="latin1")

# --- Filtrar datos desde 2016 ---
for station, df in precland_df_dict.items():
    df['Date'] = pd.to_datetime(df['Date'])
    precland_df_dict[station] = df[df['Date'] >= '2016-01-01'].reset_index(drop=True)

# --- Crear fila N (latitudes) y fila E (longitudes) ---
latitudes = ['N'] + [ubicaciones_df.loc[ubicaciones_df['Stations'] == s, 'Lat'].values[0]
                     for s in precland_df_dict.keys()]
longitudes = ['E'] + [ubicaciones_df.loc[ubicaciones_df['Stations'] == s, 'Lon'].values[0]
                      for s in precland_df_dict.keys()]

# --- Crear DataFrame final con fechas ---
all_dates = precland_df_dict[list(precland_df_dict.keys())[0]]['Date'].dt.strftime('%-m/%-d/%Y')
final_df = pd.DataFrame({'Date': all_dates})

# Agregar datos diarios de cada estación
for station, df in precland_df_dict.items():
    final_df[station] = df['prec'].values

# Combinar filas N y E al inicio
final_df = pd.concat([pd.DataFrame([latitudes, longitudes], columns=['Date'] + list(precland_df_dict.keys())),
                      final_df], ignore_index=True)

# Guardar CSV
output_file = os.path.join(output_dir, "precland_2016_2024.csv")
final_df.to_csv(output_file, index=False)

print(f"Archivo generado: {output_file}")


KeyError: 'Date'

In [43]:
print(precland_df_dict.keys())
print(len(precland_df_dict))


dict_keys([])
0


In [72]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

# Cargar datos
df = pd.read_csv(r"D:\MATUCANA.csv")

# Asegurar que la fecha esté bien parseada
df['time'] = pd.to_datetime(df['time'], dayfirst=True)

# Funciones de métricas adicionales
def nse(sim, obs):
    return 1 - np.sum((sim - obs) ** 2) / np.sum((obs - np.mean(obs)) ** 2)

def pbias(sim, obs):
    return 100 * np.sum(sim - obs) / np.sum(obs)

# Lista de modelos a evaluar
modelos = ['lgbm', 'xgb', 'rf', 'pisco',"lgbm-land"]

# Calcular métricas
resultados = {}
for modelo in modelos:
    sim = df[modelo].values
    obs = df['obs'].values
    
    rmse = mean_squared_error(obs, sim, squared=False)
    mae = mean_absolute_error(obs, sim)
    r, _ = pearsonr(obs, sim)
    r2 = r2_score(obs, sim)
    nse_val = nse(sim, obs)
    pbias_val = pbias(sim, obs)
    
    resultados[modelo] = {
        'RMSE': rmse,
        'MAE': mae,
        'PBIAS': pbias_val,
        'NSE': nse_val,
        'R': r,
        'R²': r2
    }

# Mostrar resultados en tabla
df_resultados = pd.DataFrame(resultados).T
print(df_resultados)

               RMSE       MAE       PBIAS        NSE         R         R²
lgbm       2.136552  0.888095  100.799201  -1.226628  0.499981  -1.226628
xgb        1.960302  0.884249  105.894106  -0.874420  0.549089  -0.874420
rf         1.981905  0.965934  114.352314  -0.915961  0.495527  -0.915961
pisco      0.489356  0.222161   27.072927   0.883193  0.949990   0.883193
lgbm-land  4.999685  2.859707  485.514486 -11.192877  0.340594 -11.192877
